In [48]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html
import boto3 
import datetime
import time

In [49]:
dx = boto3.client('dataexchange', region_name='us-east-1')

In [50]:
data_set_id='d0e9bd6148e8f14889980954017b0927'

In [51]:
dx.get_data_set(DataSetId='d0e9bd6148e8f14889980954017b0927')

{'ResponseMetadata': {'RequestId': 'e6e0f2fe-5acd-49ec-aa49-9c3842a57b6b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '474',
   'connection': 'keep-alive',
   'date': 'Tue, 10 Nov 2020 20:33:35 GMT',
   'x-amzn-requestid': 'e6e0f2fe-5acd-49ec-aa49-9c3842a57b6b',
   'x-amz-apigw-id': 'Vzvc9GG2oAMFrMw=',
   'x-amzn-trace-id': 'Root=1-5faaf91f-649005bf6dc930b8590abb0a',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 786d821259e050ebca76b8e357819f78.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'SEA19-C1',
   'x-amz-cf-id': 'R1hHV5Un6bMVBW3AqjIVDUxQOKZFaf6UCikGIiU0USC1tPy4V8ALBQ=='},
  'RetryAttempts': 0},
 'Arn': 'arn:aws:dataexchange:us-east-1::data-sets/d0e9bd6148e8f14889980954017b0927',
 'AssetType': 'S3_SNAPSHOT',
 'CreatedAt': datetime.datetime(2020, 3, 27, 3, 43, 31, 511000, tzinfo=tzutc()),
 'Description': 'Data for 2020, updated weekly. ',
 'Id': 'd0e9bd6148e8f14889980954017b0927',
 'Name': 'US Imports CBP Automated

In [52]:
res = dx.list_data_set_revisions(DataSetId=data_set_id)

In [53]:
def get_all_assets(data_set_id, revision_id):
    assets = []
    res = dx.list_revision_assets(DataSetId=data_set_id,
                                  RevisionId=revision_id)
    next_token = res.get('NextToken')
    
    assets += res.get('Assets')
    while next_token:
        res = dx.list_revision_assets(DataSetId=data_set_id,
                                      RevisionId=revision_id,
                                      NextToken=next_token)
        assets += res.get('Assets')
        next_token = res.get('NextToken')
        
    return assets

In [54]:
assets = get_all_assets(data_set_id, res['Revisions'][0]['Arn'].split('/')[-1])

In [55]:
def export_assets(assets, bucket):
    
    asset_destinations = []

    for asset in assets:
        asset_destinations.append({
            "AssetId": asset.get('Id'),
            "Bucket": bucket,
            "Key": asset.get('Name')
        })

    job = dx.create_job(Type='EXPORT_ASSETS_TO_S3', Details={
        "ExportAssetsToS3": {
            "RevisionId": asset.get("RevisionId"), "DataSetId": asset.get("DataSetId"),
            "AssetDestinations": asset_destinations
        }
    })

    job_id = job.get('Id')
    dx.start_job(JobId=job_id)

    while True:
        job = dx.get_job(JobId=job_id)

        if job.get('State') == 'COMPLETED':
            break
        elif job.get('State') == 'ERROR':
            raise Exception("Job {} failed to complete - {}".format(
                job_id, job.get('Errors')[0].get('Message'))
            )

        time.sleep(60)

In [56]:
export_assets(assets, 'jacky-data-engineering-us-export')